# Calculation of the Stats

## Set up

In [8]:
import pandas as pd
import os

## Read Data

In [9]:
from utils import read_stock

In [10]:
# path of the stocks folder containing the history
PATH_STOCK = os.getcwd() + "/stocks/"

# get the list of the files
files_list = [file_name for file_name in os.listdir(PATH_STOCK)]
# get the tickers
tickers = [os.path.splitext(ticker)[0] for ticker in files_list]
len(tickers)

3261

## Stats Collection

1. Daily Returns

In [15]:
def daily_return(df_stock: pd.DataFrame) -> pd.DataFrame:
    """
    Function that calculate the daily return of a stock.

    @param df_stock: df of a stock history extracted from yf.
    @return: df with the daily return column added.
    """
    df_stock["daily_return"] = (df_stock["Close"] / df_stock["Close"].shift(1)) - 1
    return df_stock

2. Cumulative Returns

In [16]:
def cum_return(df_stock: pd.DataFrame) -> pd.DataFrame:
    """
    Function that calculate the cumulatice returns of a stock.
    This function need to be executed on a df that is having the daily return column.

    @param df_stock: df of a stock history.
    @return: df with the cumulatice return column added.
    """
    df_stock["cum_return"] = (1 + df_stock["daily_return"]).cumprod()
    return df_stock

3. Bollinger Bands

In [11]:
def bollinger_bands(df_stock: pd.DataFrame,
                    period: int = 20,
                    std: float = 1.96) -> pd.DataFrame:
    """
    Function that calculate the 3 bands of the Bollinger Bands.

    @param df_stock: df of a stock history.
    @param period: period to consider to calculate the bands.
    @param std: Standard Deviation value to consider to calculate the bands.
    @return: df with the 3 values of the 3 bands. 
    """
    df_stock["middle_band"] = df_stock["Close"].rolling(window=period).mean()
    df_stock['upper_band'] = df_stock['middle_band'] + std * df_stock['Close'].rolling(window=period).std()
    df_stock['lower_band'] = df_stock['middle_band'] - std * df_stock['Close'].rolling(window=period).std()
    return df_stock

4. Ichimoku

In [18]:
def ichimoku(df_stock: pd.DataFrame,
             short_period: int = 9,
             long_period: int = 26) -> pd.DataFrame:
    """
    Function that calculate the Ichimoku indicator.

    @param df_stock: df of a stock history.
    @param short_period: int that define the short period of the indicator
    @param long_period: int that define the long period of the indicator
    @return: df with the different columns added.
    """
    high_val_short = df_stock["High"].rolling(window=short_period).max()
    low_val_short = df_stock["Low"].rolling(window=short_period).min()
    df_stock["conversion"] = (high_val_short + low_val_short) / 2

    high_val_long = df_stock["High"].rolling(window=long_period).max()
    low_val_long = df_stock["Low"].rolling(window=long_period).min()
    df_stock["baseline"] = (high_val_long + low_val_long) / 2

    df_stock["spanA"] = ((df_stock["conversion"] + df_stock["baseline"]) / 2).shift(long_period)
    high_val_2long = df_stock["High"].rolling(window=2*long_period).max()
    low_val_2long = df_stock["Low"].rolling(window=2*long_period).min()
    df_stock["spanB"] = ((high_val_2long + low_val_2long) / 2).shift(long_period)

    df_stock["lagging"] = df_stock["Close"].shift(-long_period)

    return df_stock

## Populating Stats to the ticker csv files

In [ ]:
for ticker in tickers:
    try:
        print("Calculating on: {}".format(ticker))
        df_stock_updated = read_stock(ticker, PATH_STOCK)
        df_stock_updated = daily_return(df_stock_updated)
        df_stock_updated = cum_return(df_stock_updated)
        df_stock_updated = bollinger_bands(df_stock_updated)
        df_stock_updated = ichimoku(df_stock_updated)
        df_stock_updated.to_csv(PATH_STOCK + ticker + ".csv")
        
    except Exception as e:
        print(e)